In [1]:
import spacy

In [8]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.7 MB/s eta 0:00:0000:0100:04
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
nlp = spacy.load("en_core_web_lg")

In [8]:
filename = 'iitg_annotations.json'  # Replace with your JSON file name

In [9]:
import json
tuples = []
def read_json_file(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    return data

def extract_tuples(data):

    for item in data:
        text = item[0]
        entities = item[1]['entities']
        tuples.append({'text': text, 'entities': entities})
    return tuples

def main():
    data = read_json_file(filename)
    tuples = extract_tuples(data)
    print(tuples)

if __name__ == '__main__':
    main()


[{'text': 'IITG was Established in 1994. Nestled along the picturesque northern banks of the Brahmaputra River in Guwahati, Assam, the institute offers a wide array of undergraduate, postgraduate, and doctoral programs across engineering, science, and humanities disciplines. Its sprawling 700-acre campus boasts state-of-the-art infrastructure, including academic buildings, research centers, hostels, sports facilities, and student activity centers, providing an enriching environment for learning and innovation.\r', 'entities': [[82, 93, 'HOSTEL']]}, {'text': '\r', 'entities': []}, {'text': "Beyond academics, IIT Guwahati fosters a vibrant student life with numerous clubs, societies, and cultural events that cater to a diverse range of interests, from music and dance to entrepreneurship initiatives. The institute's emphasis on research and innovation has led to significant contributions in fields such as nanotechnology, biotechnology, renewable energy, and environmental science, with a s

In [10]:
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans

nlp = spacy.blank('en')

In [11]:
doc_bin = DocBin()
for training_example in tqdm(tuples):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

100%|██████████| 19/19 [00:00<00:00, 483.15it/s]


In [12]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     45.00    0.00    0.00    0.00    0.00
 22     200         75.99    839.34   97.44  100.00   95.00    0.97
 46     400          2.24     16.11  100.00  100.00  100.00    1.00
 73     600          0.00      0.00  100.00  100.00  100.00    1.00
^C


In [13]:
nlp_ner = spacy.load("/home/raone/LLM Project/NER/model-best")

In [16]:
doc = nlp_ner("As you walk along Suryapath. you have core1 on left, to your right is Core2. Further down the road, you come across Core3.")

spacy.displacy.render(doc, style="ent")